The RDF Star program

In [ ]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, RDFS, SDO
import pprint

In [ ]:
#create a graph for the knowledge graph, for other knowledge graphs paste the file location and name within the parse function
g = Graph()
g.parse("D:\\Downloads\\AnnaTest.ttl")

#for stmt in g:
#    pprint.pprint(stmt)

<Graph identifier=Ndf35ecdfe2104378ab691ca6e5308b3f (<class 'rdflib.graph.Graph'>)>

In [ ]:
# returns all proxies for a provided person
def linkedProxies(ProvidedPerson, g):
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ns2: <http://data.biographynet.nl/rdf/>
    PREFIX idm: <http://www.intavia.eu/idm-core/>
    select distinct ?s
    where { 
        ?s idm:person_proxy_for <%s> .
    }
    """ % ProvidedPerson
    
    proxies = g.query(query)
    return proxies

In [ ]:
# returns biodes source for a proxy
def findSource(Proxy, g):
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ns2: <http://data.biographynet.nl/rdf/>
    PREFIX idm: <http://www.intavia.eu/idm-core/>
    PREFIX ore: <http://www.openarchives.org/ore/terms/>
    select ?biodes
    where { 
        <%s> ore:proxyIn ?biodes .
    }
    """ % Proxy
    
    Biodes = g.query(query)
    for stmt in  Biodes:
        return(stmt.biodes)

In [ ]:
#returns all triples connected to proxy to depth of 2
def allTriples(Proxy, g): 
    # create a graph for a description
    # add all triples with subject that matches the description
    new_graph = Graph()
    new_graph += g.triples((Proxy, None, None))
    for stmt in g.triples((Proxy, None, None)):
        subject = stmt[2]
        new_graph += g.triples((subject, None, None))
    return new_graph

In [25]:
#for triples with proxy as subject, we can now use the actual person
def replaceProxyByPerson(triple, Person, Proxy):
    s, p, o = triple
    if s == Proxy:
        s = Person
    new_triple = s, p, o
    return new_triple

In [30]:
# for one provided person, write all rdf star statements
def PersonToRDFSTAR(ProvidedPerson, g):
    for row in linkedProxies(ProvidedPerson, g):
        Proxy = row['s']
        Source = findSource(Proxy, g)
        all_triples = allTriples(Proxy, g)
        for triple in all_triples:
            triple = replaceProxyByPerson(triple, ProvidedPerson, Proxy)
            s, p, o = triple
            f.write("<< <" + str(s) +"> <" + str(p) + "> <" + str(o) + "> >> <http://example.org/saidBy> <" + Source + "> .\n")

In [31]:
#make a query to find all the persons in the knowledge graph
def find_persons(g):
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ns2: <http://data.biographynet.nl/rdf/>
    PREFIX idm: <http://www.intavia.eu/idm-core/>
    select distinct ?person
    where { 
        ?person rdf:type idm:Provided_Person .
    }
    """

    persons = g.query(query)
    return persons

In [32]:
Persons = find_persons(g)
with open('D:\\rdf_star_anna1.ttls', 'a') as f:
    for row in Persons:
        Person = row['person']
        PersonToRDFSTAR(Person, g)

<< <http://data.biographynet.nl/rdf/Person-99999999> <http://data.biographynet.nl/rdf/has_faith> <http://data.biographynet.nl/rdf/Faith-Rooms-Katholiek> >> <http://example.org/saidBy> <http://data.biographynet.nl/rdf/BioDes-99999999_01 does not look like a valid URI, trying to serialize this will break.
<< <http://data.biographynet.nl/rdf/Person-99999999> <http://data.biographynet.nl/rdf/has_faith> <http://data.biographynet.nl/rdf/Faith-Rooms-Katholiek> >> <http://example.org/saidBy> <http://data.biographynet.nl/rdf/BioDes-99999999_01> .
 does not look like a valid URI, trying to serialize this will break.
<< <http://data.biographynet.nl/rdf/Person-99999999> <http://ldf.fi/schema/bioc/has_nationality> <http://ldf.fi/schema/bioc/dutch> >> <http://example.org/saidBy> <http://data.biographynet.nl/rdf/BioDes-99999999_01 does not look like a valid URI, trying to serialize this will break.
<< <http://data.biographynet.nl/rdf/Person-99999999> <http://ldf.fi/schema/bioc/has_nationality> <http: